# Problem definition
![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

# The data
## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

# Imports and initial exploration

In [12]:
import pandas as pd
import numpy as np

In [13]:
# Read in data
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


# Data preparation

## Client data

In [14]:
# Create new data frame

columns = [
    'client_id',
    'age',
    'job',
    'marital',
    'education',
    'credit_default',
    'mortgage'
]
df_client = df[columns]

# For "job", change "." to "_"
df_client.job = df_client.job.str.replace('.', '_')

# For "education", change "." to "_" and "unknown" to np.NaN
df_client.education = df_client.education.str.replace('.', '_')
df_client.loc[df_client.education == 'unknown', 'education'] = np.nan

# For "credit_default" and "mortgage", convert to boolean data type: 1 if "yes", otherwise 0
for c in ['credit_default', 'mortgage']:
    df_client[c] = df_client[c].apply(lambda x: 1 if x == 'yes' else 0)
    df_client[c] = df_client[c].astype('bool')

# Check data types
print(df_client.dtypes)

# Export to csv an display dataframe
df_client.to_csv('client.csv', index=False)
df_client

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object


,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,False,True
41184,41184,46,blue-collar,married,professional_course,False,False
41185,41185,56,retired,married,university_degree,False,True
41186,41186,44,technician,married,professional_course,False,False


## Campaign data

In [15]:
# Add 'year' to df and create a date string containing the day, month, and year
df['year'] = 2022
df['date_str'] = df.apply(lambda x: f'{x.day} {x.month} {x.year}', axis=1)
df['last_contact_date'] = pd.to_datetime(df.date_str, format='%d %b %Y')

# Create new data frame
columns = [
    'client_id',
    'number_contacts',
    'contact_duration',
    'previous_campaign_contacts',
    'previous_outcome',
    'campaign_outcome',
    'last_contact_date'
]
df_campaign = df[columns]

# For "previous_outcome" and "campaign_outcome", convert to boolean data type: 1 if "success" or "yes", otherwise 0.
for c in ['previous_outcome', 'campaign_outcome']:
    df_campaign[c] = df_campaign[c].apply(lambda x: 1 if x in ['yes', 'success'] else 0)
    df_campaign[c] = df_campaign[c].astype('bool')

# Check data types
print(df_campaign.dtypes)

# Export to csv an display dataframe
df_campaign.to_csv('campaign.csv', date_format='%Y-%m-%d', index=False)
df_campaign

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                        bool
campaign_outcome                        bool
last_contact_date             datetime64[ns]
dtype: object


,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03
...,...,...,...,...,...,...,...
41183,41183,1,334,0,False,True,2022-11-30
41184,41184,1,383,0,False,False,2022-11-06
41185,41185,2,189,0,False,False,2022-11-24
41186,41186,1,442,0,False,True,2022-11-17


## Economics data

In [16]:
# Create new data frame
columns = [
    'client_id',
    'cons_price_idx',
    'euribor_three_months'
]
df_econ = df[columns]
df_econ

# Check data types
print(df_econ.dtypes)

# Export to csv an display dataframe
df_econ.to_csv('economics.csv', index=False)
df_econ

client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object


,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857
...,...,...,...
41183,41183,94.767,1.028
41184,41184,94.767,1.028
41185,41185,94.767,1.028
41186,41186,94.767,1.028
